In [5]:
# Core
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning / Stats
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [6]:
df = pd.read_csv('/Applications/WorkDataSets/Data Analysis/Oracle Data.csv')

In [7]:
# Check which columns you have first
df.columns.tolist()

['party_number',
 'email_address',
 'person_name',
 'party_unique_name',
 'department_code',
 'do_not_contact_flag',
 'do_not_email_flag',
 'do_not_mail_flag',
 'job_title',
 'job_title_code',
 'company_name',
 'account_type',
 'country',
 'state',
 'city',
 'CustomerId_NAR',
 'CustomerName_NAR',
 'account_segmentation',
 'total_arr',
 'ats_owner',
 'ae_owner',
 'GEO']

In [8]:
contacts_per_nar = (
    df.groupby('CustomerId_NAR')['person_name']
      .count()
      .reset_index(name='contact_count')
      .sort_values(by='contact_count', ascending=False)
)

contacts_per_nar.head(10)

,CustomerId_NAR,contact_count
11788,A-CIT-3,2095
62175,H-446796,818
62609,H-CIT-45057803,728
63435,H-CIT-45849839,705
65062,H-CIT-52731901,618
62770,H-CIT-45200029,579
62425,H-CIT-24364000,574
64344,H-CIT-50245214,550
62694,H-CIT-45149679,536
62611,H-CIT-45058453,524


In [15]:
# 1. Filter to only enterprise accounts
enterprise_df = df[df['account_segmentation'] == 'CITRIX_ENTERPRISE']

# 2. Replace blanks with NaN for consistency
enterprise_df = enterprise_df.replace('', pd.NA)

# 3. Create the breakdown pivot
permission_pivot = (
    enterprise_df
    .groupby(['CustomerId_NAR', 'CustomerName_NAR', 'GEO'])
    .agg(
        contact_count=('person_name', 'count'),
        total_arr_sum=('total_arr', 'sum'),
        do_not_email_yes=('do_not_email_flag', lambda x: (x == 'Y').sum()),
        do_not_email_no=('do_not_email_flag', lambda x: (x == 'N').sum()),
        do_not_email_null=('do_not_email_flag', lambda x: x.isna().sum()),
        do_not_contact_yes=('do_not_contact_flag', lambda x: (x == 'Y').sum()),
        do_not_contact_no=('do_not_contact_flag', lambda x: (x == 'N').sum()),
        do_not_contact_null=('do_not_contact_flag', lambda x: x.isna().sum())
    )
    .reset_index()
    .sort_values(by='contact_count', ascending=False)
)

# 4. Display summary
print("Enterprise Accounts in dataset:", permission_pivot.shape[0])
display(permission_pivot.head(10))

Enterprise Accounts in dataset: 2997


,CustomerId_NAR,CustomerName_NAR,GEO,contact_count,total_arr_sum,do_not_email_yes,do_not_email_no,do_not_email_null,do_not_contact_yes,do_not_contact_no,do_not_contact_null
18,H-446796,Volkswagen AG,CE DACH IND DEVELOP ADM1,818,4.018437e+08,538,280,0,74,744,0
452,H-CIT-45057803,Microsoft,ISV North America ADM1,728,1.209596e+10,230,498,0,64,664,0
1278,H-CIT-45849839,Bechtle Aktiengesellschaft MSP,MSP Europe AE3,705,0.000000e+00,446,259,0,69,636,0
2905,H-CIT-52731901,Nokia Oyj ISV,ISV Europe AE1,618,4.645420e+07,283,335,0,9,609,0
613,H-CIT-45200029,Netherlands Government,WE NL GROW AE1,579,7.855952e+08,389,190,0,47,532,0
268,H-CIT-24364000,Deutsche Telekom/T-Systems ES,CE DACH IND DEVELOP ADM2,563,5.385760e+07,250,313,0,70,493,0
2187,H-CIT-50245214,FUJITSU LIMITED GSI,GSI SPM1,550,3.988994e+07,277,273,0,62,488,0
537,H-CIT-45149679,United Healthcare Group,NA ENT GROW-D AE2,536,1.002927e+09,209,327,0,45,491,0
454,H-CIT-45058453,Thomson Reuters,NA ENT DEVELOP ADM2,524,5.980343e+08,259,265,0,2,522,0
2504,H-CIT-51177728,Factset Research Systems Inc.,ISV North America ADM1,494,5.163380e+07,314,180,0,3,491,0


In [16]:
# 1. Filter to only CITRIX_ENTERPRISE accounts
enterprise_df = df[df['account_segmentation'] == 'CITRIX_ENTERPRISE'].copy()

# 2. Normalize blanks to NaN for consistent null detection
enterprise_df = enterprise_df.replace('', pd.NA)

# 3. Create the per-account permission + email completeness summary
permission_pivot = (
    enterprise_df
    .groupby(['CustomerId_NAR', 'CustomerName_NAR', 'GEO'])
    .agg(
        contact_count=('person_name', 'count'),
        total_arr_sum=('total_arr', 'sum'),

        # Do Not Email breakdowns
        do_not_email_yes=('do_not_email_flag', lambda x: (x == 'Y').sum()),
        do_not_email_no=('do_not_email_flag', lambda x: (x == 'N').sum()),
        do_not_email_null=('do_not_email_flag', lambda x: x.isna().sum()),

        # Do Not Contact breakdowns
        do_not_contact_yes=('do_not_contact_flag', lambda x: (x == 'Y').sum()),
        do_not_contact_no=('do_not_contact_flag', lambda x: (x == 'N').sum()),
        do_not_contact_null=('do_not_contact_flag', lambda x: x.isna().sum()),

        # Missing email addresses
        missing_email_count=('email_address', lambda x: x.isna().sum())
    )
    .reset_index()
    .sort_values(by='contact_count', ascending=False)
)

# 4. Calculate % of contacts missing email (for quick insight)
permission_pivot['missing_email_pct'] = (
    permission_pivot['missing_email_count'] / permission_pivot['contact_count'] * 100
).round(1)

# 5. Print high-level stats
total_accounts = enterprise_df['CustomerId_NAR'].nunique()
total_contacts = len(enterprise_df)

print("──────────────────────────────")
print(f"Total Enterprise Contacts: {total_contacts:,}")
print(f"Unique Enterprise NAR Accounts: {total_accounts:,}")
print("──────────────────────────────\n")

print("Top 10 Enterprise Accounts by Contact Count:")
display(permission_pivot.head(10))

──────────────────────────────
Total Enterprise Contacts: 107,458
Unique Enterprise NAR Accounts: 2,997
──────────────────────────────

Top 10 Enterprise Accounts by Contact Count:


,CustomerId_NAR,CustomerName_NAR,GEO,contact_count,total_arr_sum,do_not_email_yes,do_not_email_no,do_not_email_null,do_not_contact_yes,do_not_contact_no,do_not_contact_null,missing_email_count,missing_email_pct
18,H-446796,Volkswagen AG,CE DACH IND DEVELOP ADM1,818,4.018437e+08,538,280,0,74,744,0,42,5.1
452,H-CIT-45057803,Microsoft,ISV North America ADM1,728,1.209596e+10,230,498,0,64,664,0,22,3.0
1278,H-CIT-45849839,Bechtle Aktiengesellschaft MSP,MSP Europe AE3,705,0.000000e+00,446,259,0,69,636,0,25,3.5
2905,H-CIT-52731901,Nokia Oyj ISV,ISV Europe AE1,618,4.645420e+07,283,335,0,9,609,0,296,47.9
613,H-CIT-45200029,Netherlands Government,WE NL GROW AE1,579,7.855952e+08,389,190,0,47,532,0,22,3.8
268,H-CIT-24364000,Deutsche Telekom/T-Systems ES,CE DACH IND DEVELOP ADM2,563,5.385760e+07,250,313,0,70,493,0,42,7.5
2187,H-CIT-50245214,FUJITSU LIMITED GSI,GSI SPM1,550,3.988994e+07,277,273,0,62,488,0,40,7.3
537,H-CIT-45149679,United Healthcare Group,NA ENT GROW-D AE2,536,1.002927e+09,209,327,0,45,491,0,21,3.9
454,H-CIT-45058453,Thomson Reuters,NA ENT DEVELOP ADM2,524,5.980343e+08,259,265,0,2,522,0,6,1.1
2504,H-CIT-51177728,Factset Research Systems Inc.,ISV North America ADM1,494,5.163380e+07,314,180,0,3,491,0,2,0.4


In [17]:
# 1. Filter to only CITRIX_ENTERPRISE accounts
enterprise_df = df[df['account_segmentation'] == 'CITRIX_ENTERPRISE'].copy()

# 2. Normalize blanks to NaN
enterprise_df = enterprise_df.replace('', pd.NA)

# 3. Build pivot table with all relevant breakdowns
permission_pivot = (
    enterprise_df
    .groupby(['CustomerId_NAR', 'CustomerName_NAR', 'GEO'])
    .agg(
        contact_count=('person_name', 'count'),
        total_arr_sum=('total_arr', 'sum'),

        # Do Not Email breakdowns
        do_not_email_yes=('do_not_email_flag', lambda x: (x == 'Y').sum()),
        do_not_email_no=('do_not_email_flag', lambda x: (x == 'N').sum()),
        do_not_email_null=('do_not_email_flag', lambda x: x.isna().sum()),

        # Do Not Contact breakdowns
        do_not_contact_yes=('do_not_contact_flag', lambda x: (x == 'Y').sum()),
        do_not_contact_no=('do_not_contact_flag', lambda x: (x == 'N').sum()),
        do_not_contact_null=('do_not_contact_flag', lambda x: x.isna().sum()),

        # Missing email addresses
        missing_email_count=('email_address', lambda x: x.isna().sum()),

        # Job title stats
        unique_job_titles=('job_title', pd.Series.nunique),
        missing_job_title=('job_title', lambda x: x.isna().sum()),

        # Job title code stats
        unique_job_title_codes=('job_title_code', pd.Series.nunique),
        missing_job_title_code=('job_title_code', lambda x: x.isna().sum())
    )
    .reset_index()
)

# 4. Calculate percentages for data completeness
permission_pivot['missing_email_pct'] = (
    permission_pivot['missing_email_count'] / permission_pivot['contact_count'] * 100
).round(1)

permission_pivot['missing_job_title_pct'] = (
    permission_pivot['missing_job_title'] / permission_pivot['contact_count'] * 100
).round(1)

permission_pivot['missing_job_title_code_pct'] = (
    permission_pivot['missing_job_title_code'] / permission_pivot['contact_count'] * 100
).round(1)

# 5. Sort and print summary
permission_pivot = permission_pivot.sort_values(by='contact_count', ascending=False)

total_accounts = enterprise_df['CustomerId_NAR'].nunique()
total_contacts = len(enterprise_df)

print("──────────────────────────────")
print(f"Total Enterprise Contacts: {total_contacts:,}")
print(f"Unique Enterprise NAR Accounts: {total_accounts:,}")
print("──────────────────────────────\n")

print("Top 10 Enterprise Accounts by Contact Count:")
display(permission_pivot.head(10))

──────────────────────────────
Total Enterprise Contacts: 107,458
Unique Enterprise NAR Accounts: 2,997
──────────────────────────────

Top 10 Enterprise Accounts by Contact Count:


,CustomerId_NAR,CustomerName_NAR,GEO,contact_count,total_arr_sum,do_not_email_yes,do_not_email_no,do_not_email_null,do_not_contact_yes,do_not_contact_no,do_not_contact_null,missing_email_count,unique_job_titles,missing_job_title,unique_job_title_codes,missing_job_title_code,missing_email_pct,missing_job_title_pct,missing_job_title_code_pct
18,H-446796,Volkswagen AG,CE DACH IND DEVELOP ADM1,818,4.018437e+08,538,280,0,74,744,0,42,108,638,15,486,5.1,78.0,59.4
452,H-CIT-45057803,Microsoft,ISV North America ADM1,728,1.209596e+10,230,498,0,64,664,0,22,70,600,12,556,3.0,82.4,76.4
1278,H-CIT-45849839,Bechtle Aktiengesellschaft MSP,MSP Europe AE3,705,0.000000e+00,446,259,0,69,636,0,25,72,516,12,234,3.5,73.2,33.2
2905,H-CIT-52731901,Nokia Oyj ISV,ISV Europe AE1,618,4.645420e+07,283,335,0,9,609,0,296,29,341,11,331,47.9,55.2,53.6
613,H-CIT-45200029,Netherlands Government,WE NL GROW AE1,579,7.855952e+08,389,190,0,47,532,0,22,238,269,13,154,3.8,46.5,26.6
268,H-CIT-24364000,Deutsche Telekom/T-Systems ES,CE DACH IND DEVELOP ADM2,563,5.385760e+07,250,313,0,70,493,0,42,94,383,12,176,7.5,68.0,31.3
2187,H-CIT-50245214,FUJITSU LIMITED GSI,GSI SPM1,550,3.988994e+07,277,273,0,62,488,0,40,91,407,13,209,7.3,74.0,38.0
537,H-CIT-45149679,United Healthcare Group,NA ENT GROW-D AE2,536,1.002927e+09,209,327,0,45,491,0,21,111,352,10,280,3.9,65.7,52.2
454,H-CIT-45058453,Thomson Reuters,NA ENT DEVELOP ADM2,524,5.980343e+08,259,265,0,2,522,0,6,39,443,9,422,1.1,84.5,80.5
2504,H-CIT-51177728,Factset Research Systems Inc.,ISV North America ADM1,494,5.163380e+07,314,180,0,3,491,0,2,68,332,8,300,0.4,67.2,60.7


In [18]:
# 1. Filter to only CITRIX_ENTERPRISE accounts
enterprise_df = df[df['account_segmentation'] == 'CITRIX_ENTERPRISE'].copy()

# 2. Normalize blanks to NaN
enterprise_df = enterprise_df.replace('', pd.NA)

# 3. Build pivot table with all relevant breakdowns
permission_pivot = (
    enterprise_df
    .groupby(['CustomerId_NAR', 'CustomerName_NAR', 'GEO'])
    .agg(
        contact_count=('person_name', 'count'),
        total_arr_sum=('total_arr', 'sum'),

        # Do Not Email breakdowns
        do_not_email_yes=('do_not_email_flag', lambda x: (x == 'Y').sum()),
        do_not_email_no=('do_not_email_flag', lambda x: (x == 'N').sum()),
        do_not_email_null=('do_not_email_flag', lambda x: x.isna().sum()),

        # Do Not Contact breakdowns
        do_not_contact_yes=('do_not_contact_flag', lambda x: (x == 'Y').sum()),
        do_not_contact_no=('do_not_contact_flag', lambda x: (x == 'N').sum()),
        do_not_contact_null=('do_not_contact_flag', lambda x: x.isna().sum()),

        # Missing email addresses
        missing_email_count=('email_address', lambda x: x.isna().sum()),

        # Job title stats
        unique_job_titles=('job_title', pd.Series.nunique),
        missing_job_title=('job_title', lambda x: x.isna().sum()),

        # Job title code stats
        unique_job_title_codes=('job_title_code', pd.Series.nunique),
        missing_job_title_code=('job_title_code', lambda x: x.isna().sum())
    )
    .reset_index()
)

# 4. Calculate percentages for data completeness
permission_pivot['missing_email_pct'] = (
    permission_pivot['missing_email_count'] / permission_pivot['contact_count'] * 100
).round(1)

permission_pivot['missing_job_title_pct'] = (
    permission_pivot['missing_job_title'] / permission_pivot['contact_count'] * 100
).round(1)

permission_pivot['missing_job_title_code_pct'] = (
    permission_pivot['missing_job_title_code'] / permission_pivot['contact_count'] * 100
).round(1)

# 5. Sort and print summary
permission_pivot = permission_pivot.sort_values(by='contact_count', ascending=False)

total_accounts = enterprise_df['CustomerId_NAR'].nunique()
total_contacts = len(enterprise_df)

print("──────────────────────────────")
print(f"Total Enterprise Contacts: {total_contacts:,}")
print(f"Unique Enterprise NAR Accounts: {total_accounts:,}")
print("──────────────────────────────\n")

print("Top 10 Enterprise Accounts by Contact Count:")
display(permission_pivot.head(10))

──────────────────────────────
Total Enterprise Contacts: 107,458
Unique Enterprise NAR Accounts: 2,997
──────────────────────────────

Top 10 Enterprise Accounts by Contact Count:


,CustomerId_NAR,CustomerName_NAR,GEO,contact_count,total_arr_sum,do_not_email_yes,do_not_email_no,do_not_email_null,do_not_contact_yes,do_not_contact_no,do_not_contact_null,missing_email_count,unique_job_titles,missing_job_title,unique_job_title_codes,missing_job_title_code,missing_email_pct,missing_job_title_pct,missing_job_title_code_pct
18,H-446796,Volkswagen AG,CE DACH IND DEVELOP ADM1,818,4.018437e+08,538,280,0,74,744,0,42,108,638,15,486,5.1,78.0,59.4
452,H-CIT-45057803,Microsoft,ISV North America ADM1,728,1.209596e+10,230,498,0,64,664,0,22,70,600,12,556,3.0,82.4,76.4
1278,H-CIT-45849839,Bechtle Aktiengesellschaft MSP,MSP Europe AE3,705,0.000000e+00,446,259,0,69,636,0,25,72,516,12,234,3.5,73.2,33.2
2905,H-CIT-52731901,Nokia Oyj ISV,ISV Europe AE1,618,4.645420e+07,283,335,0,9,609,0,296,29,341,11,331,47.9,55.2,53.6
613,H-CIT-45200029,Netherlands Government,WE NL GROW AE1,579,7.855952e+08,389,190,0,47,532,0,22,238,269,13,154,3.8,46.5,26.6
268,H-CIT-24364000,Deutsche Telekom/T-Systems ES,CE DACH IND DEVELOP ADM2,563,5.385760e+07,250,313,0,70,493,0,42,94,383,12,176,7.5,68.0,31.3
2187,H-CIT-50245214,FUJITSU LIMITED GSI,GSI SPM1,550,3.988994e+07,277,273,0,62,488,0,40,91,407,13,209,7.3,74.0,38.0
537,H-CIT-45149679,United Healthcare Group,NA ENT GROW-D AE2,536,1.002927e+09,209,327,0,45,491,0,21,111,352,10,280,3.9,65.7,52.2
454,H-CIT-45058453,Thomson Reuters,NA ENT DEVELOP ADM2,524,5.980343e+08,259,265,0,2,522,0,6,39,443,9,422,1.1,84.5,80.5
2504,H-CIT-51177728,Factset Research Systems Inc.,ISV North America ADM1,494,5.163380e+07,314,180,0,3,491,0,2,68,332,8,300,0.4,67.2,60.7


In [19]:
# 6. Export to CSV
output_path = '/Applications/WorkDataSets/Data Analysis/Enterprise_Contact_Quality.csv'
permission_pivot.to_csv(output_path, index=False)

print(f"✅ Data successfully exported to:\n{output_path}")

✅ Data successfully exported to:
/Applications/WorkDataSets/Data Analysis/Enterprise_Contact_Quality.csv


In [20]:
summary_stats = {
    'total_accounts': permission_pivot['CustomerId_NAR'].nunique(),
    'avg_contacts_per_account': permission_pivot['contact_count'].mean().round(2),
    'median_contacts_per_account': permission_pivot['contact_count'].median(),
    'max_contacts_in_account': permission_pivot['contact_count'].max(),
    'avg_total_arr': permission_pivot['total_arr_sum'].mean().round(2),
    'avg_missing_email_pct': permission_pivot['missing_email_pct'].mean().round(1),
    'avg_missing_job_title_pct': permission_pivot['missing_job_title_pct'].mean().round(1),
    'avg_missing_job_title_code_pct': permission_pivot['missing_job_title_code_pct'].mean().round(1),
    'avg_unique_job_titles': permission_pivot['unique_job_titles'].mean().round(2),
    'avg_unique_job_title_codes': permission_pivot['unique_job_title_codes'].mean().round(2),
}

for k, v in summary_stats.items():
    print(f"{k:35}: {v}")

total_accounts                     : 2997
avg_contacts_per_account           : 35.86
median_contacts_per_account        : 17.0
max_contacts_in_account            : 818
avg_total_arr                      : 109537483.74
avg_missing_email_pct              : 6.1
avg_missing_job_title_pct          : 63.1
avg_missing_job_title_code_pct     : 39.4
avg_unique_job_titles              : 9.16
avg_unique_job_title_codes         : 4.71


In [21]:
# 1. Filter to enterprise accounts
enterprise_df = df[df['account_segmentation'] == 'CITRIX_ENTERPRISE'].copy()
enterprise_df = enterprise_df.replace('', pd.NA)

# 2. Define helper masks
do_not_email_yes = (enterprise_df['do_not_email_flag'] == 'Y')
do_not_contact_yes = (enterprise_df['do_not_contact_flag'] == 'Y')

# 3. Compute overall counts
total_contacts = len(enterprise_df)
email_optouts = do_not_email_yes.sum()
contact_optouts = do_not_contact_yes.sum()
both_optouts = (do_not_email_yes & do_not_contact_yes).sum()

# 4. Compute percentages
summary = pd.DataFrame({
    'Metric': [
        'Total Enterprise Contacts',
        'Do Not Email = Y',
        'Do Not Contact = Y',
        'Both Do Not Email + Contact = Y'
    ],
    'Count': [
        total_contacts,
        email_optouts,
        contact_optouts,
        both_optouts
    ],
    'Percent of Total': [
        100.0,
        round(email_optouts / total_contacts * 100, 2),
        round(contact_optouts / total_contacts * 100, 2),
        round(both_optouts / total_contacts * 100, 2)
    ]
})

# 5. Display the summary
print("───────────────────────────────────────")
print("Global Opt-Out Summary (Enterprise Accounts)")
print("───────────────────────────────────────")
display(summary)

───────────────────────────────────────
Global Opt-Out Summary (Enterprise Accounts)
───────────────────────────────────────


,Metric,Count,Percent of Total
0,Total Enterprise Contacts,107458,100.00
1,Do Not Email = Y,50337,46.84
2,Do Not Contact = Y,8068,7.51
3,Both Do Not Email + Contact = Y,8068,7.51


In [23]:
# Filter to enterprise accounts
enterprise_df = df[df['account_segmentation'] == 'CITRIX_ENTERPRISE'].copy()

# Normalize flags to clean up inconsistent entries
norm_map = {'TRUE':'Y','T':'Y','YES':'Y','FALSE':'N','F':'N','NO':'N'}
for col in ['do_not_email_flag', 'do_not_contact_flag']:
    enterprise_df[col] = (enterprise_df[col]
                          .astype(str).str.strip().str.upper()
                          .replace({'': pd.NA, 'NONE': pd.NA, 'NAN': pd.NA})
                          .replace(norm_map))

# Define masks
email_y   = (enterprise_df['do_not_email_flag'] == 'Y')
contact_y = (enterprise_df['do_not_contact_flag'] == 'Y')
either_y  = (email_y | contact_y)    # ✅ anyone who opted out of *either*

# Compute counts
total_contacts   = len(enterprise_df)
email_optouts    = email_y.sum()
contact_optouts  = contact_y.sum()
both_optouts     = (email_y & contact_y).sum()
either_optouts   = either_y.sum()

# Build summary
summary = pd.DataFrame({
    'Metric': [
        'Total Enterprise Contacts',
        'Do Not Email = Y',
        'Do Not Contact = Y',
        'Both Do Not Email + Contact = Y',
        'Either (Email OR Contact) = Y'   # ✅ combined opt-out metric
    ],
    'Count': [
        total_contacts,
        email_optouts,
        contact_optouts,
        both_optouts,
        either_optouts
    ],
    'Percent of Total': [
        100.00,
        round(email_optouts / total_contacts * 100, 2),
        round(contact_optouts / total_contacts * 100, 2),
        round(both_optouts / total_contacts * 100, 2),
        round(either_optouts / total_contacts * 100, 2)   # ✅ combined %
    ]
})

display(summary)

,Metric,Count,Percent of Total
0,Total Enterprise Contacts,107458,100.00
1,Do Not Email = Y,50337,46.84
2,Do Not Contact = Y,8068,7.51
3,Both Do Not Email + Contact = Y,8068,7.51
4,Either (Email OR Contact) = Y,50337,46.84


In [24]:
# Check unique flag combinations to confirm overlap
combo_counts = (
    enterprise_df
    .assign(
        email_flag=enterprise_df['do_not_email_flag'],
        contact_flag=enterprise_df['do_not_contact_flag']
    )
    .groupby(['email_flag', 'contact_flag'])
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)
display(combo_counts)

,email_flag,contact_flag,count
0,N,N,57121
1,Y,N,42269
2,Y,Y,8068


In [27]:
job_title_summary = (
    enterprise_df.groupby('job_title')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)

print("Top 20 Job Titles:")
display(job_title_summary.head(100))

Top 20 Job Titles:


,job_title,count
12641,System Administrator,956
6067,IT Manager,900
12750,System Or Network Administrator,540
2062,Consultant,512
3971,Engineer,461
...,...,...
659,Assistant,30
9459,Principal Technology Manager,30
9454,Principal Systems Engineer,29
12847,Systems Administrator II,29


In [26]:
job_code_summary = (
    enterprise_df.groupby('job_title_code')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)

print("Top 20 Job Title Codes:")
display(job_code_summary.head(20))

Top 20 Job Title Codes:


,job_title_code,count
24,SYSTEM_ADMINISTRATOR,15751
18,NETWORK_ADMINISTRATOR_ENGINEER,11015
14,IT_MANAGER,9959
10,DIRECTOR,6119
7,CONSULTANT,4386
1,ARCHITECT,3739
8,C_LEVEL_PRESIDENT_VP,3726
3,BUSINESS_MANAGER,2075
0,ANALYST,2053
19,OTHER,1939


In [28]:
total_contacts = len(enterprise_df)
missing_title = enterprise_df['job_title'].isna().sum()
missing_code = enterprise_df['job_title_code'].isna().sum()

print("──────────────────────────────")
print(f"Total contacts: {total_contacts:,}")
print(f"Missing job titles: {missing_title:,} ({missing_title / total_contacts:.1%})")
print(f"Missing job codes: {missing_code:,} ({missing_code / total_contacts:.1%})")
print("──────────────────────────────")

──────────────────────────────
Total contacts: 107,458
Missing job titles: 68,867 (64.1%)
Missing job codes: 45,550 (42.4%)
──────────────────────────────


In [29]:
import pandas as pd
from datetime import datetime

# --- 1. Job Title Breakdown
job_title_summary = (
    enterprise_df.groupby('job_title')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)
job_title_summary['section'] = 'Job Titles'

# --- 2. Job Title Code Breakdown
job_code_summary = (
    enterprise_df.groupby('job_title_code')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)
job_code_summary.rename(columns={'job_title_code': 'job_title'}, inplace=True)
job_code_summary['section'] = 'Job Title Codes'

# --- 4. Missing Data Summary (for titles + codes)
total_contacts = len(enterprise_df)
missing_title = enterprise_df['job_title'].isna().sum()
missing_code = enterprise_df['job_title_code'].isna().sum()

missing_summary = pd.DataFrame({
    'job_title': ['Missing Job Titles', 'Missing Job Title Codes'],
    'count': [missing_title, missing_code],
    'percent_missing': [
        round(missing_title / total_contacts * 100, 2),
        round(missing_code / total_contacts * 100, 2)
    ],
    'section': ['Missing Data', 'Missing Data']
})

# --- Combine all into one export
combined_export = pd.concat(
    [job_title_summary, job_code_summary, missing_summary],
    ignore_index=True
)

# --- Save to CSV
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
output_path = f"/Applications/WorkDataSets/Data Analysis/Job_Title_Breakdown_{timestamp}.csv"

combined_export.to_csv(output_path, index=False)

print(f"✅ Combined job title breakdown exported to:\n{output_path}")

✅ Combined job title breakdown exported to:
/Applications/WorkDataSets/Data Analysis/Job_Title_Breakdown_2025-10-07_07-37.csv


In [31]:


# --- Filter to only CITRIX_ENTERPRISE accounts ---
enterprise_df = df[df['account_segmentation'] == 'CITRIX_ENTERPRISE'].copy()

# --- Summarize job title codes only ---
job_code_breakdown = (
    enterprise_df
    .groupby('job_title_code')
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)

# --- Add missing info for completeness ---
total_contacts = len(enterprise_df)
missing_codes = enterprise_df['job_title_code'].isna().sum()
valid_codes = total_contacts - missing_codes

# --- Add summary row at bottom ---
summary_row = pd.DataFrame({
    'job_title_code': ['(Missing Codes)', '(Total Records)'],
    'count': [missing_codes, total_contacts]
})

job_code_breakdown_export = pd.concat([job_code_breakdown, summary_row], ignore_index=True)

# --- Export to CSV ---
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
output_path = f"/Applications/WorkDataSets/Data Analysis/Job_Code_Breakdown_{timestamp}.csv"

job_code_breakdown_export.to_csv(output_path, index=False)

print(f"✅ Job title code breakdown exported to:\n{output_path}")

✅ Job title code breakdown exported to:
/Applications/WorkDataSets/Data Analysis/Job_Code_Breakdown_2025-10-07_07-57.csv
